<a href="https://colab.research.google.com/github/ttn9171/Portfolio/blob/main/Modeling/UC2%3A%20NBA/Feature%20Extraction%20and%20Engineer/GA_Feature_Extraction_p1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Setting up connection

In [ ]:
!apt-get update
!apt-get install -y -q curl gnupg

!curl -fsSL https://packages.cloud.google.com/apt/doc/apt-key.gpg | gpg --dearmor -o /usr/share/keyrings/cloud.google.gpg
!echo "deb [signed-by=/usr/share/keyrings/cloud.google.gpg] https://packages.cloud.google.com/apt gcsfuse-jammy main" | tee /etc/apt/sources.list.d/gcsfuse.list
!apt-get update

!apt-get install -y -q gcsfuse

from google.colab import auth
auth.authenticate_user()

import os
project_id = 'capstone-aldo'
os.environ['Aldo_Capstone'] = project_id

from google.cloud import storage

client = storage.Client(project=project_id)
bucket = 'mma-capstone'

os.makedirs('/content/gcs', exist_ok=True)
!gcsfuse {bucket} /content/gcs

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 https://packages.cloud.google.com/apt gcsfuse-jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
R

In [ ]:
import pandas as pd
import numpy as np
import dask.dataframe as dd

In [ ]:
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)

## Get only visitors who click on email at least once

In [ ]:
# prompt: how can i combine df_visit_counts and df_visit_counts_US, they have same columns. i want to group them by fullvisitor_id, keep only the first 4 columns, and sum the values in the columns
df_visit_counts_CA = dd.read_csv('/content/gcs/data_preprocessed/visit_counts_CA_2023.csv')
df_visit_counts_US = dd.read_csv('/content/gcs/data_preprocessed/visit_counts_US_2023.csv')

df_visit_counts_CA['country'] = 'ALDO_CA'
df_visit_counts_US['country'] = 'ALDO_US'

# Combine df_visit_counts and df_visit_counts_US
df_combined = dd.concat([df_visit_counts_CA, df_visit_counts_US])
df_combined = df_combined.drop(columns=['avg_interactions_per_session'], axis=1)


# Compute the result
df_visitcounts = df_combined.compute()
df_visitcounts = df_visitcounts.reset_index()

df_visitcounts['avg_interactions_per_session'] = df_visitcounts['total_interactions'] / df_visitcounts['total_visits']
df_visitcounts['email_visit_rate'] = df_visitcounts['email_visits'] / df_visitcounts['total_visits']
df_visitcounts.to_parquet('/content/gcs/data_preprocessed/visit_counts_2023.parquet')

email_visitor_list = df_visitcounts[(df_visitcounts['email_visits'] > 0)][['fullvisitor_id', 'country']].drop_duplicates()
email_visitor_list.to_parquet('/content/gcs/data_preprocessed/visitor_list_country.parquet')

In [ ]:
df_visitcounts[df_visitcounts['email_visits'] > 0]

,index,fullvisitor_id,total_visits,email_visits,total_interactions,country,avg_interactions_per_session,email_visit_rate
22,22,=1000013274624227492=,1,1,34,ALDO_CA,34.000000,1.0000
107,107,=10000635762438619988=,16,1,68,ALDO_CA,4.250000,0.0625
127,127,=100007722476408233=,24,3,223,ALDO_CA,9.291667,0.1250
133,133,=1000078602261103680=,2,2,3,ALDO_CA,1.500000,1.0000
144,144,=1000083029201305882=,1,1,5,ALDO_CA,5.000000,1.0000
...,...,...,...,...,...,...,...,...
22724273,2137715,=999941428418636627=,1,1,1,ALDO_US,1.000000,1.0000
22724288,2137730,=9999475349804604430=,1,1,4,ALDO_US,4.000000,1.0000
22724289,2137731,=9999476445320704563=,1,1,12,ALDO_US,12.000000,1.0000
22724311,2137753,=999954592490162830=,2,1,3,ALDO_US,1.500000,0.5000


## Get Session Events (add to cart, view product, etc) from High Response Group


In [ ]:
df_session_event_CA = dd.read_csv('/content/gcs/data_preprocessed/session_event_CA_2023.csv')
df_session_event_US = dd.read_csv('/content/gcs/data_preprocessed/session_event_US_2023.csv')

df_session_event = dd.concat([df_session_event_CA, df_session_event_US])

# Compute the merged DataFrame
df_session_event = df_session_event.compute()
df_session_event.drop_duplicates(inplace=True)

In [ ]:
import dask.dataframe as dd
import pandas as pd

def process_session_event_high_response(ca_path, us_path, visitor_list_path, output_path):
    # 1. Load CA and US session_event files
    df_ca = dd.read_csv(ca_path)
    df_us = dd.read_csv(us_path)

    # 2. Combine and drop duplicates
    df_session_event = dd.concat([df_ca, df_us], axis=0)
    df_session_event = df_session_event.drop_duplicates()

    # 3. Load visitor list (assumed small)
    visitor_list = pd.read_parquet(visitor_list_path)

    # 4. Ensure matching dtypes for merge keys
    for col in ['fullvisitor_id', 'country']:
        df_session_event[col] = df_session_event[col].astype('object')
        visitor_list[col] = visitor_list[col].astype('object')

    # 5. Filter only high-response visitors
    df_session_event_high_response = df_session_event.merge(
        visitor_list[['fullvisitor_id', 'country']],
        on=['fullvisitor_id', 'country'],
        how='inner'
    )

    # 6. Compute and save
    df_result = df_session_event_high_response.compute()
    df_result.to_parquet(output_path, index=False)
    print(f"✅ Saved high-response session events to: {output_path}")

process_session_event_high_response(
    ca_path="/content/gcs/data_preprocessed/session_event_CA_2023.csv",
    us_path="/content/gcs/data_preprocessed/session_event_US_2023.csv",
    visitor_list_path="/content/gcs/data_preprocessed/visitor_list_country.parquet",
    output_path="/content/gcs/data_preprocessed/session_event_high_response_2023.parquet"
)



/usr/local/lib/python3.11/dist-packages/dask/dataframe/multi.py:521: UserWarning: Merging dataframes with merge column data type mismatches: 
+--------------------------------------+------------+-------------+
| Merge columns                        | left dtype | right dtype |
+--------------------------------------+------------+-------------+
| ('fullvisitor_id', 'fullvisitor_id') | object     | string      |
| ('country', 'country')               | object     | string      |
+--------------------------------------+------------+-------------+
Cast dtypes explicitly to avoid unexpected results.
  warnings.warn(


✅ Saved high-response session events to: /content/gcs/data_preprocessed/session_event_high_response_2023.parquet


## Getting page path related info

In [ ]:
import dask.dataframe as dd
import pandas as pd

# Define shared columns to keep from page_paths
COLUMNS_TO_KEEP = ['fullvisitor_id', 'unique_session_id', 'date', 'is_email_traffic', 'traffic_source', 'utm_category', 'product_id']

def process_page_path_file(page_path_path, visitor_list_path, country_label):
    # Load page path data
    df_page_path = dd.read_parquet(page_path_path, columns=COLUMNS_TO_KEEP)
    df_page_path['country'] = country_label
    df_page_path['country'] = df_page_path['country'].astype(str)

    # Load visitor list
    visitor_list = pd.read_parquet(visitor_list_path)
    visitor_list['country'] = visitor_list['country'].astype(str)

    # Filter by matching fullvisitor_id and country
    df_filtered = df_page_path.merge(
        visitor_list[['fullvisitor_id', 'country']],
        on=['fullvisitor_id', 'country'],
        how='inner'
    )

    # Convert 'date' column to datetime
    df_filtered = df_filtered.map_partitions(
        lambda df: df.assign(date=pd.to_datetime(df['date']))
    )

    # Group and aggregate main metrics
    grouped_df = (
        df_filtered.groupby(['fullvisitor_id', 'unique_session_id', 'date'])
        .agg({
            'country': 'first',
            'traffic_source': 'first',
            'utm_category': 'first',
            'is_email_traffic': 'max'
        })
        .reset_index()
    )

    # Count unique product views
    product_counts = (
        df_filtered.groupby(['fullvisitor_id', 'unique_session_id', 'date'])['product_id']
        .nunique()
        .reset_index()
        .rename(columns={'product_id': 'unique_product_view_counts'})
    )

    # Merge all features together
    final_df = grouped_df.merge(product_counts, on=['fullvisitor_id', 'unique_session_id', 'date'], how='left')

    return final_df


def full_pagepath_processing(ca_path, us_path, visitor_list_path, output_path_ca, output_path_us, output_path_combined):
    # Process and save CA
    df_ca = process_page_path_file(ca_path, visitor_list_path, 'ALDO_CA')
    df_ca_result = df_ca.compute()
    df_ca_result.to_parquet(output_path_ca, index=False)
    print(f"Saved CA file to: {output_path_ca}")

    # Process and save US
    df_us = process_page_path_file(us_path, visitor_list_path, 'ALDO_US')
    df_us_result = df_us.compute()
    df_us_result.to_parquet(output_path_us, index=False)
    print(f"Saved US file to: {output_path_us}")

    # Combine and save merged file
    df_combined = pd.concat([df_ca_result, df_us_result], ignore_index=True).drop_duplicates()
    df_combined.to_parquet(output_path_combined, index=False)
    print(f"Saved combined file to: {output_path_combined}")


full_pagepath_processing(
    ca_path="/content/gcs/data_preprocessed/page_paths_CA_2023.parquet",
    us_path="/content/gcs/data_preprocessed/page_paths_US_2023.parquet",
    visitor_list_path="/content/gcs/data_preprocessed/visitor_list_country.parquet",
    output_path_ca="/content/gcs/data_preprocessed/page_paths_high_response_CA_2023.parquet",
    output_path_us="/content/gcs/data_preprocessed/page_paths_high_response_US_2023.parquet",
    output_path_combined="/content/gcs/data_preprocessed/page_paths_high_response_combined_2023.parquet"
)


/usr/local/lib/python3.11/dist-packages/dask/dataframe/multi.py:521: UserWarning: Merging dataframes with merge column data type mismatches: 
+------------------------+------------+-------------+
| Merge columns          | left dtype | right dtype |
+------------------------+------------+-------------+
| ('country', 'country') | object     | string      |
+------------------------+------------+-------------+
Cast dtypes explicitly to avoid unexpected results.
  warnings.warn(


Saved CA file to: /content/gcs/data_preprocessed/page_paths_high_response_CA_2023.parquet


/usr/local/lib/python3.11/dist-packages/dask/dataframe/multi.py:521: UserWarning: Merging dataframes with merge column data type mismatches: 
+------------------------+------------+-------------+
| Merge columns          | left dtype | right dtype |
+------------------------+------------+-------------+
| ('country', 'country') | object     | string      |
+------------------------+------------+-------------+
Cast dtypes explicitly to avoid unexpected results.
  warnings.warn(


Saved US file to: /content/gcs/data_preprocessed/page_paths_high_response_US_2023.parquet
Saved combined file to: /content/gcs/data_preprocessed/page_paths_high_response_combined_2023.parquet
